In [190]:
import pandas as pd
import numpy as np

In [198]:
df = pd.read_excel(r'/Users/claireekohler/Downloads/73461-PL_JBC_2024_11_15.xlsx')

In [200]:
print(df)

         Job Phase  Segment    Job-Ph-Seg  S/S Asgn Firm Asgn Div Task ID  \
0    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    E200   
1    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    P300   
2    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    P902   
3    73461.0    BL      1.0  73461-BL-001  1.0        HI       HG    E200   
4    73461.0    BL      1.0  73461-BL-001  1.0        HI       HG    P300   
..       ...   ...      ...           ...  ...       ...      ...     ...   
369  73461.0    PL     31.0  73461-PL-031  1.0        HI       NW    E999   
370  73461.0    PL     31.0  73461-PL-031  1.0        HI       NW    T901   
371  73461.0    PL     31.0  73461-PL-031  2.0        HI       NW    E005   
372  73461.0    PL     31.0  73461-PL-031  3.0        HI       NW    C999   
373      NaN   NaN      NaN           NaN  NaN       NaN      NaN     NaN   

     Task Asgn Ofc S/S S/L  ...                   Segment Title  \
0       

In [202]:
hours_index = df.columns.get_loc('Hours')
print(hours_index)

19


In [204]:
#Add 'Adjustment' and 'New Hours' Columns

df.insert(hours_index + 1, 'Adjustment', 0)
df.insert(hours_index + 2, 'New Hours',0)

In [206]:
#Check that the columns have been added
print(df[['Hours','Adjustment','New Hours']])

     Hours  Adjustment  New Hours
0        0           0          0
1       45           0          0
2       10           0          0
3        0           0          0
4        5           0          0
..     ...         ...        ...
369      0           0          0
370     88           0          0
371      0           0          0
372     84           0          0
373   6950           0          0

[374 rows x 3 columns]


In [208]:
# Convert 'Job-Ph-Seg' and 'S/S' columns to string type
df['Job-Ph-Seg'] = df['Job-Ph-Seg'].astype(str)
df['S/S'] = df['S/S'].astype(str)

In [210]:
df.dtypes

Job                              float64
Phase                             object
Segment                          float64
Job-Ph-Seg                        object
S/S                               object
Asgn Firm                         object
Asgn Div                          object
Task ID                           object
Task Asgn Ofc                    float64
S/S S/L                           object
Task Assign Name                  object
PM Name                           object
Owning Firm                       object
Owning Div                        object
PM Ofc                           float64
Task Status                       object
Task Type                         object
Task Description                  object
Baseline ER                      float64
Hours                              int64
Adjustment                         int64
New Hours                          int64
Rate                             float64
Base Cost                          int64
Bud ER          

In [212]:
#Function to add hours to 'Adjustment' and 'New Hours'

#s/s cant be 999 or 997

def cover_hours(df):
    # Group the DataFrame by 'Job-Ph-Seg'
    grouped = df.groupby('Job-Ph-Seg')

    # Loop through each group
    for name, group in grouped:
        for idx, row in group.iterrows():
            # Check if 'Job-Ph-Seg' contains 'PL' and 'S/S' does not contain '999'
            if 'PL' in row['Job-Ph-Seg'] and '999' not in row['S/S']:
                # Get the value of 'Actual Remain Hours'
                actual_remaining_hours = row['Actual Remain Hours']
                
                # If 'Actual Remain Hours' is negative
                if actual_remaining_hours < 0:
                    # Calculate the adjustment as |Actual Remaining Hours| + 1
                    X = abs(actual_remaining_hours) + 1
                    
                    # Set 'Adjustment' to X and update 'New Hours'
                    df.at[idx, 'Adjustment'] = X
                    df.at[idx, 'New Hours'] = row['Hours'] + X     
    return df

In [214]:
#Function to balance hours

def balance_hours(df):
    #Group by 'Job-Ph-Seg' to handle each Ph-Seg independently
    grouped = df.groupby('Job-Ph-Seg')

    #Loop through each group
    for name, group in grouped:
        #Track sum of adjustments for current group
        total_adjustments = group['Adjustment'].sum()
        
        #Loop through each row in current segment
        for idx, row in group.iterrows():
            if total_adjustments == 0:
                break
            #Consider rows only where adjusted
            adjustment = row['Adjustment']
            if adjustment > 0:
            #Find rows in same segment with 'Actual Remain Hours' greater than 'Adjustment'
                eligible_rows = group[(group['Actual Remain Hours'] > adjustment) & (group['Adjustment'] == 0)]
                for target_idx in eligible_rows.index:
                   target_remain_hours = df.at[target_idx, 'Actual Remain Hours']
                   df.at[target_idx, 'Adjustment'] = -adjustment
                   df.at[target_idx,'New Hours'] = df.at[target_idx, 'Hours'] - adjustment

                total_adjustments -= adjustment

                if total_adjustments == 0:
                    break
    return df


In [216]:
#Add 'new loaded costs remaining' and 'net' columns

loaded_costs_remain_index = df.columns.get_loc('Loaded Costs Remaining')

#print(loaded_costs_remain_index)

df.insert(loaded_costs_remain_index + 1, 'New Loaded Costs Remaining', 0)
df.insert(loaded_costs_remain_index + 2, 'NET',0)


In [218]:
#Check that the columns have been added and are next to "LCR'
new_LCR_index = df.columns.get_loc('New Loaded Costs Remaining')
NET_index = df.columns.get_loc('NET')

print(new_LCR_index)
print(NET_index)

36
37


In [220]:
#Function to calculate 'New Loaded Costs' Remaining and 'NET'

#if 'new hours' isnt null,
#insert into 'new loaded costs remaining' = (new hours*rate*budget ER)-loaded costs remaining
#net = adjustment*rate*budgeted ER

def new_loaded_costs(df):
    # Group by 'Job-Ph-Seg' to handle each Ph-Seg independently
    grouped = df.groupby('Job-Ph-Seg')

    # Loop through each group
    for name, group in grouped:
        # Loop through each row in the current segment
        for idx, row in group.iterrows():
            # Consider only rows with a non-null 'Adjustment'
            adjustment = row['Adjustment']
            if adjustment != 0:
                # Calculate 'New Loaded Costs Remaining'
                df.at[idx, 'New Loaded Costs Remaining'] = (
                    df.at[idx, 'New Hours'] * df.at[idx, 'Bud ER'] * df.at[idx, 'Rate']
                    - df.at[idx, 'Loaded JTD Costs']
                )
                # Calculate 'NET'
                df.at[idx, 'NET'] = (
                    df.at[idx, 'Adjustment'] * df.at[idx, 'Bud ER'] * df.at[idx, 'Rate']
                )

    return df



In [222]:
df.dtypes

Job                                  float64
Phase                                 object
Segment                              float64
Job-Ph-Seg                            object
S/S                                   object
Asgn Firm                             object
Asgn Div                              object
Task ID                               object
Task Asgn Ofc                        float64
S/S S/L                               object
Task Assign Name                      object
PM Name                               object
Owning Firm                           object
Owning Div                            object
PM Ofc                               float64
Task Status                           object
Task Type                             object
Task Description                      object
Baseline ER                          float64
Hours                                  int64
Adjustment                             int64
New Hours                              int64
Rate      

In [224]:
#Apply functions to dataframe and view output

df = cover_hours(df)

In [226]:
df = balance_hours(df)
df = new_loaded_costs(df)

/var/folders/mj/rcxzfd7n4lqg7cq1g15cy3mc0000gn/T/ipykernel_90559/1008790020.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2736.9771530816706' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'New Loaded Costs Remaining'] = (
/var/folders/mj/rcxzfd7n4lqg7cq1g15cy3mc0000gn/T/ipykernel_90559/1008790020.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1698.5737861145026' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'NET'] = (


In [228]:
print(df)

         Job Phase  Segment    Job-Ph-Seg  S/S Asgn Firm Asgn Div Task ID  \
0    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    E200   
1    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    P300   
2    73461.0    BL      1.0  73461-BL-001  1.0        HI       NW    P902   
3    73461.0    BL      1.0  73461-BL-001  1.0        HI       HG    E200   
4    73461.0    BL      1.0  73461-BL-001  1.0        HI       HG    P300   
..       ...   ...      ...           ...  ...       ...      ...     ...   
369  73461.0    PL     31.0  73461-PL-031  1.0        HI       NW    E999   
370  73461.0    PL     31.0  73461-PL-031  1.0        HI       NW    T901   
371  73461.0    PL     31.0  73461-PL-031  2.0        HI       NW    E005   
372  73461.0    PL     31.0  73461-PL-031  3.0        HI       NW    C999   
373      NaN   NaN      NaN           nan  nan       NaN      NaN     NaN   

     Task Asgn Ofc S/S S/L  ...                   Segment Title  \
0       

In [230]:
df.to_excel('JBC_73461-11-20.xlsx', index=False)